# Paramaterized Runs for Multiple Brain Training

This notebook aims to templatize a process for creating alterations of a Bonsai experiment, and submit all of them to the Bonsai platform.

## Supported operations:

1. Adjust goal objective weights
2. Adjust any training parameters, including adding new ones
3. Adjust some algorithm parameters such as memory
4. Create inkling files from alterations
5. Submit all brain-versions to platform and start training
6. Query status of brains
7. Assess all brains with same configuration file

## NOT yet implemented:

1. Adjust experimental tags
2. Adjust algorithm network
3. ~~changing training parameters for specific lessons~~
4. Remove/add goals
5. Adjust actual goal ranges or values

## Necessary Imports

In [1]:
from batch_creation import *
from batch_containers import *
from ink_parser import *

In [2]:
import pandas as pd
import datetime

## Read Inkling File

We need a base inkling file, which we will parse and modify using provided functions in the `ink_parser` script. We recommend adding double hash marks `##` to break up important components of your inkling file. This will make it easier to parse the file for important components. 

In [17]:
cartpole_ink = "cartpole_single_concept.ink"

In [18]:
cartpole_ink_str = read_ink(cartpole_ink)

## Adjust Goal Objective Weights

In [19]:
goal_pri = adjust_weight_objectives(cartpole_ink_str, {"FallOver": 2})

## Adjust Training Parameters

In [20]:
train_params = {
	"EpisodeIterationLimit": 100,
	"TotalIterationLimit": 10 ** 7,
	"NoProgressIterationLimit": 10 ** 5,
	"LessonAssessmentWindow": 50,
	# "LessonRewardThreshold": 200,
	"LessonSuccessThreshold": 0.95,
}
goal_pri_tp = adjust_training_params(
    goal_pri, train_params, "## concept-level training params"
)

## Adjust Algorithm Parameters

In [31]:
algo_params = {
    "MemoryMode": '"state and action"',
    "Algorithm": '"PPO"',
    "BatchSize": 6000
}

In [32]:
goal_pri_tp_memory = adjust_algorithm_params(goal_pri_tp, algo_params)

In [33]:
ppo_cp = adjust_algorithm_params(cartpole_ink_str, algo_params)

In [29]:
goal_pri_ppo_memory = adjust_algorithm_params(cartpole_ink_str, algo_params)

## Save to New Inkling Files

In [34]:
write_ink("\n".join(cp_memory), "cp_memory.ink")
write_ink("\n".join(goal_pri_ppo_memory), "cartpole-ppo-bs.ink")

In [9]:
write_ink("\n".join(goal_pri_tp), "cp_goalpri_tp.ink")
write_ink("\n".join(goal_pri_tp_memory), "cp_goalwpri_tp_memory.ink")

## Launch Training with Each New Inkling
Create a list of all the Inkling files you want to train from.

In [10]:
inkling_files = [cartpole_ink, "cp_memory.ink", "cp_goalwpri.ink", "cp_goalpri_tp.ink", "cp_goalwpri_tp_memory.ink"]

## Launch Parameters

In [11]:
NUM_RUNS = 4
SIM_PACKAGE = "mba-pole"
EXPERIMENT_NAME = "cartpole-launch-test"
SCALE_PLATFORM = "ACI"
NUM_INSTANCES = 20
LOW_PRI_NODES = 10
DEDICATED_NODES = 0
CONCEPT_NAME = "BalancePole"


In [12]:
running_experiments = pd.DataFrame({})

In [13]:
for i, INK_FILE in enumerate(inkling_files):
#     SIM_NAME = f"{EXPERIMENT_NAME}_pool{i}"
#     TASK = f"python main.py --sim-name {SIM_NAME}" 
    BRAIN_NAME = INK_FILE.replace(".ink", "")

    for i in range(1, NUM_RUNS + 1):
        TASK = f"python main.py"
        SIM_NAME = SIM_PACKAGE
        BRAIN_VERSION = str(i)
        POOL_NAME = EXPERIMENT_NAME + str(i)
        status = run_sims_connect(
            config_file = "configs/userconfig.ini",
            task_to_run=TASK,
            scale_platform=SCALE_PLATFORM,
            num_instances=NUM_INSTANCES,
            brain_name=BRAIN_NAME,
            brain_version=BRAIN_VERSION,
            pool_name=POOL_NAME,
            # only needed for batch experiments
            low_pri_nodes=LOW_PRI_NODES,
            dedicated_nodes=DEDICATED_NODES,
            sim_name = SIM_NAME,
            ink_file=INK_FILE,
            wait=False,
            connect_sims_brain=False,
            sleep_time=0,
        )
        
        ## save experiment dictionary for tracking status later
        running_experiments = running_experiments.append({
            "experiment": EXPERIMENT_NAME,
            "brain": BRAIN_NAME,
            "version": BRAIN_VERSION,
            "concept_name": CONCEPT_NAME,
            "inkling": INK_FILE,
            "scale_platform": SCALE_PLATFORM,
            "low_pri_nodes": LOW_PRI_NODES,
            "dedicated_nodes": DEDICATED_NODES,
            "sim": SIM_NAME,
            "pool_name": POOL_NAME,
            "task": TASK,
            "experiment_start_time": datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S"),
            "last_checked": datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S"),
            "status": status,
        }, ignore_index=True)

/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)
/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)
/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


...

/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


Copied cp_goalwpri_tp_memory version 3 to version 4.


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


In [15]:
running_experiments

,brain,concept_name,dedicated_nodes,experiment,experiment_start_time,inkling,last_checked,low_pri_nodes,pool_name,scale_platform,sim,status,task,version
0,cartpole_single_concept,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-11-14,cartpole_single_concept.ink,2022-02-09-17-11-14,10.0,cartpole-launch-test1,ACI,mba-pole,Active,python main.py,1
1,cartpole_single_concept,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-11-58,cartpole_single_concept.ink,2022-02-09-17-11-58,10.0,cartpole-launch-test2,ACI,mba-pole,Active,python main.py,2
2,cartpole_single_concept,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-12-45,cartpole_single_concept.ink,2022-02-09-17-12-45,10.0,cartpole-launch-test3,ACI,mba-pole,Active,python main.py,3
3,cartpole_single_concept,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-13-32,cartpole_single_concept.ink,2022-02-09-17-13-32,10.0,cartpole-launch-test4,ACI,mba-pole,Active,python main.py,4
4,cp_memory,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-13-46,cp_memory.ink,2022-02-09-17-13-46,10.0,cartpole-launch-test1,ACI,mba-pole,Active,python main.py,1
5,cp_memory,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-14-32,cp_memory.ink,2022-02-09-17-14-32,10.0,cartpole-launch-test2,ACI,mba-pole,Active,python main.py,2
6,cp_memory,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-15-19,cp_memory.ink,2022-02-09-17-15-19,10.0,cartpole-launch-test3,ACI,mba-pole,Active,python main.py,3
7,cp_memory,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-16-05,cp_memory.ink,2022-02-09-17-16-05,10.0,cartpole-launch-test4,ACI,mba-pole,Active,python main.py,4
8,cp_goalwpri,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-16-21,cp_goalwpri.ink,2022-02-09-17-16-21,10.0,cartpole-launch-test1,ACI,mba-pole,Active,python main.py,1
9,cp_goalwpri,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-17-08,cp_goalwpri.ink,2022-02-09-17-17-08,10.0,cartpole-launch-test2,ACI,mba-pole,Active,python main.py,2


## Check Status of Brain and Brain Version

In [20]:
running_experiments["status"] = running_experiments.apply(
    lambda x: get_brain_status(brain_name=x['brain'], brain_version=x['version'], sleep_time=0),
    axis=1
)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


...

/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:61: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


In [21]:
running_experiments

,brain,concept_name,dedicated_nodes,experiment,experiment_start_time,inkling,last_checked,low_pri_nodes,pool_name,scale_platform,sim,status,task,version
0,cartpole_single_concept,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-11-14,cartpole_single_concept.ink,2022-02-09-17-11-14,10.0,cartpole-launch-test1,ACI,mba-pole,Idle,python main.py,1
1,cartpole_single_concept,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-11-58,cartpole_single_concept.ink,2022-02-09-17-11-58,10.0,cartpole-launch-test2,ACI,mba-pole,Idle,python main.py,2
2,cartpole_single_concept,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-12-45,cartpole_single_concept.ink,2022-02-09-17-12-45,10.0,cartpole-launch-test3,ACI,mba-pole,Idle,python main.py,3
3,cartpole_single_concept,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-13-32,cartpole_single_concept.ink,2022-02-09-17-13-32,10.0,cartpole-launch-test4,ACI,mba-pole,Idle,python main.py,4
4,cp_memory,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-13-46,cp_memory.ink,2022-02-09-17-13-46,10.0,cartpole-launch-test1,ACI,mba-pole,Idle,python main.py,1
5,cp_memory,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-14-32,cp_memory.ink,2022-02-09-17-14-32,10.0,cartpole-launch-test2,ACI,mba-pole,Active,python main.py,2
6,cp_memory,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-15-19,cp_memory.ink,2022-02-09-17-15-19,10.0,cartpole-launch-test3,ACI,mba-pole,Idle,python main.py,3
7,cp_memory,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-16-05,cp_memory.ink,2022-02-09-17-16-05,10.0,cartpole-launch-test4,ACI,mba-pole,Idle,python main.py,4
8,cp_goalwpri,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-16-21,cp_goalwpri.ink,2022-02-09-17-16-21,10.0,cartpole-launch-test1,ACI,mba-pole,Idle,python main.py,1
9,cp_goalwpri,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-17-08,cp_goalwpri.ink,2022-02-09-17-17-08,10.0,cartpole-launch-test2,ACI,mba-pole,Idle,python main.py,2


## Launch Assessments

In [23]:
# running_experiments.to_csv("running_experiments.csv")

In [3]:
running_experiments = pd.read_csv("running_experiments.csv")

In [4]:
running_experiments

,Unnamed: 0,brain,concept_name,dedicated_nodes,experiment,experiment_start_time,inkling,last_checked,low_pri_nodes,pool_name,scale_platform,sim,status,task,version
0,0,cartpole_single_concept,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-11-14,cartpole_single_concept.ink,2022-02-09-17-11-14,10.0,cartpole-launch-test1,ACI,mba-pole,Idle,python main.py,1
1,1,cartpole_single_concept,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-11-58,cartpole_single_concept.ink,2022-02-09-17-11-58,10.0,cartpole-launch-test2,ACI,mba-pole,Idle,python main.py,2
2,2,cartpole_single_concept,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-12-45,cartpole_single_concept.ink,2022-02-09-17-12-45,10.0,cartpole-launch-test3,ACI,mba-pole,Idle,python main.py,3
3,3,cartpole_single_concept,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-13-32,cartpole_single_concept.ink,2022-02-09-17-13-32,10.0,cartpole-launch-test4,ACI,mba-pole,Idle,python main.py,4
4,4,cp_memory,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-13-46,cp_memory.ink,2022-02-09-17-13-46,10.0,cartpole-launch-test1,ACI,mba-pole,Idle,python main.py,1
5,5,cp_memory,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-14-32,cp_memory.ink,2022-02-09-17-14-32,10.0,cartpole-launch-test2,ACI,mba-pole,Active,python main.py,2
6,6,cp_memory,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-15-19,cp_memory.ink,2022-02-09-17-15-19,10.0,cartpole-launch-test3,ACI,mba-pole,Idle,python main.py,3
7,7,cp_memory,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-16-05,cp_memory.ink,2022-02-09-17-16-05,10.0,cartpole-launch-test4,ACI,mba-pole,Idle,python main.py,4
8,8,cp_goalwpri,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-16-21,cp_goalwpri.ink,2022-02-09-17-16-21,10.0,cartpole-launch-test1,ACI,mba-pole,Idle,python main.py,1
9,9,cp_goalwpri,BalancePole,0.0,cartpole-launch-test,2022-02-09-17-17-08,cp_goalwpri.ink,2022-02-09-17-17-08,10.0,cartpole-launch-test2,ACI,mba-pole,Idle,python main.py,2


In [15]:
assessment_files = ["cartpole-assess-default.json", "cartpole-assess-initvelocity.json", "cartpole-assess-initposition.json"]
EPISODE_LIMIT=100
NUM_INSTANCES=2
for i in assessment_files:
    running_experiments.apply(
        lambda x: run_assessment(
            brain_name=x['brain'], brain_version=x['version'], concept_name=x['concept_name'],
            assess_file=i, 
            assess_name=str(datetime.datetime.now().strftime("%Y%m%d%H%M%S")+x['brain']+str(x['version'])+i).replace(".json", "")[:63],
            episode_limit=EPISODE_LIMIT, sim_package=x['sim'], num_instances=NUM_INSTANCES),
        axis=1
    )

/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)
/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)
/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)
/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)
/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


...

/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)
/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)
/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)
/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)
/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


## Get All Assessments

In [6]:
a = get_assessments("cartpole_single_concept", 1)

/anaconda/envs/bonsai-batch/lib/python3.7/site-packages/msal_extensions/token_cache.py:96: RuntimeWarning: You are using an unprotected token cache
  warnings.warn("You are using an unprotected token cache", RuntimeWarning)


In [8]:
expand_assess = pd.json_normalize(a['value'])

In [9]:
expand_assess

,assessmentName,status,description
0,automatic-09February2022-17.12.27Z,Completed,Run on 09February2022-17.12.27 UTC
1,automatic-09February2022-17.15.23Z,Completed,Run on 09February2022-17.15.23 UTC
2,automatic-09February2022-17.21.21Z,Completed,Run on 09February2022-17.21.21 UTC
3,automatic-09February2022-17.27.10Z,Completed,Run on 09February2022-17.27.10 UTC
4,cartpole_single_concept1cartpole-assess-defaul...,Completed,
5,20220209183235cartpole_single_concept1cartpole...,Completed,
6,20220209183437cartpole_single_concept1cartpole...,Completed,
7,20220209183637cartpole_single_concept1cartpole...,Completed,
8,20220209190456cartpole_single_concept1cartpole...,Completed,


In [11]:
expand_assess['assessmentName'].values

array(['automatic-09February2022-17.12.27Z',
       'automatic-09February2022-17.15.23Z',
       'automatic-09February2022-17.21.21Z',
       'automatic-09February2022-17.27.10Z',
       'cartpole_single_concept1cartpole-assess-default.json',
       '20220209183235cartpole_single_concept1cartpole-assess-default',
       '20220209183437cartpole_single_concept1cartpole-assess-initveloc',
       '20220209183637cartpole_single_concept1cartpole-assess-initposit',
       '20220209190456cartpole_single_concept1cartpole-assess-default'],
      dtype=object)